# Train Soccer Player Detector

---

[![Download Dataset](https://app.roboflow.com/images/download-dataset-badge.svg)](https://universe.roboflow.com/roboflow-jvuqo/football-players-detection-3zvbc)

## Configure API keys

- Open your [`Roboflow Settings`](https://app.roboflow.com/settings/api) page. Click `Copy`. This will place your private key in the clipboard.
- In Colab, go to the left pane and click on `Secrets` (🔑). Store Roboflow API Key under the name `ROBOFLOW_API_KEY`.

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install dependencies

In [ ]:
!pip install -q ultralytics roboflow
!pip install google-cloud-secret-manager

## Imports

In [4]:
from roboflow import Roboflow
from google.colab import userdata
from IPython.display import Image
from google.cloud import secretmanager


## Pull dataset

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

client = secretmanager.SecretManagerServiceClient()
response = client.access_secret_version(request={"name": "projects/707857790047/secrets/ROBOFLOW_API_KEY/versions/latest"})

ROBOFLOW_API_KEY = response.payload.data.decode("UTF-8")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(12)
dataset = version.download("yolov11")

NOTE: Dataset contains 4 classes: 'ball', 'goalkeeper', 'player', 'referee'. We want to ignore 'ball' class. To do it, make changes in `data.yaml`. Change values `nc` to `3` and `names` to `['goalkeeper', 'player', 'referee']`. Then run the script below.

In [6]:
import os


datasets_dirs = os.listdir(f"{HOME}/datasets")
datasets_dirs = [os.path.abspath(dir) for dir in datasets_dirs]

datasests_to_clear = ["test", "train", "valid"]

for datasets_dir in datasets_dirs:
  for dataset_to_clear in datasests_to_clear:
    for file_name in os.listdir(f"{datasets_dir}/{dataset_to_clear}/labels"):
      file_path = f"{datasets_dir}/{dataset_to_clear}/labels/{file_name}"

      with open(file_path, "r+") as f:
        lines = f.readlines()
        f.seek(0)
        f.truncate(0)
        for line in lines:
          if line.startswith("0"):
              continue

          processed_line = " ".join(
              str(int(float(value)) - 1) if i == 0 else value
              for i, value in enumerate(line.split())
          )
          f.write(processed_line + "\n")

**NOTE:** Update `data.yaml` structure to align with the latest requirements of the `ultralytics` package.



In [7]:
!sed -i 's|\(train: \).*|\1../train/images|' {dataset.location}/data.yaml
!sed -i 's|\(val: \).*|\1../valid/images|' {dataset.location}/data.yaml

## Custom training

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolo11x.pt data={dataset.location}/data.yaml batch=5 epochs=100 imgsz=1280 plots=True

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate custom model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=1280